In [11]:
#input a vector corresponding to a probability distribution and output an index according to that distribution
function markovrand(nextv)
    if round(sum(nextv), digits = 6) != 1.0
        error(nextv, " is not a stochastic vector")
    end
    maxstates = length(nextv) - 1
    r = rand(Float64)
    current = maxstates
    for i = 0:maxstates
        if r <= nextv[i + 1]
            current = i
            break
        else
            r = r - nextv[i + 1]
        end
    end
    return(current)
end


markovrand (generic function with 1 method)

In [12]:
struct AugMatrix
    mat::Matrix
    init::Vector
end

struct AugDblMatrix
    mat1::Matrix
    mat2::Matrix
    init::Vector
end

In [13]:
#output bass notes in 2*$beats notes
function bassgen(beats::Int, trans::AugMatrix)
    transmat = trans.mat
    iv = trans.init
    nextv = iv
    L = []
    for j = 1:beats
        current = markovrand(nextv)
#print(current, " ")
        bincurrent = digits(current, base = 2, pad = 2)
#print(bincurrent, "\n")
        append!(L, [bincurrent[2], bincurrent[1]])
        nextv = transmat[:, current + 1]
    end
    return(L)
end

bassgen (generic function with 1 method)

In [87]:
#0 = nobass, 1 = bass
                #00    #01   #10  #11
rockBassMatrix = [0.1e0 1//3 1//3 1; #00
                0 1//3 0 0;          #01
                0.45e0 1//3 1//3 0;  #10
                0.45e0 0 1//3 0]     #11

4×4 Matrix{Float64}:
 0.1   0.333333  0.333333  1.0
 0.0   0.333333  0.0       0.0
 0.45  0.333333  0.333333  0.0
 0.45  0.0       0.333333  0.0

In [15]:
rockBassInitial = [0.1e0,0,0.5e0,0.4e0]

4-element Vector{Float64}:
 0.1
 0.0
 0.5
 0.4

In [16]:
rockBass = AugMatrix(rockBassMatrix, rockBassInitial)

AugMatrix([0.1 0.3333333333333333 0.3333333333333333 1.0; 0.0 0.3333333333333333 0.0 0.0; 0.45 0.3333333333333333 0.3333333333333333 0.0; 0.45 0.0 0.3333333333333333 0.0], [0.1, 0.0, 0.5, 0.4])

In [17]:
#test
bassgen(4, rockBass)

8-element Vector{Any}:
 1
 0
 1
 0
 1
 1
 0
 0

In [18]:
function lookup16ride()::Matrix
    r = Matrix(undef, 4, 4)
    r[1,1] = ["N[^e","[","N[^e","[","N[c^e","[","N[^e","[","N[^e","[","N[^e","[","N[c^e","[","N[^e","["] #hihat eighths, trash
    #NB Windows midi doesn't have a good way of making a trash hat?  N defined as tenuto mark in header.abc
    r[2,1] = ["[^e", "[^e", "[^e", "[^e", "[c^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[c^e", "[^e", "[^e", "[^e"]; #hihat 16
    r[3,1] = ["!>![^e", "U[^e", "!>![^e", "U[^e", "!>![c^e", "U[^e","!>![^e", "U[^e","!>![^e", "U[^e", "!>![^e", "U[^e", "!>![c^e", "U[^e", "!>![^e", "U[^e"]; #drive 16
    r[4,1] = ["[^e", "[^e", "[^e", "[^e", "[c^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[c^e", "[^e", "[^e", "[_e"];  #one slide 16
    r[1,2] = ["[^g","[","[^g","[","[c^g","[","[^g","[","[^g","[","[^g","[","[c^g","[","[^g","["] #ride quarter
    r[2,2] = ["[^g", "[^g", "[^g", "[^g", "[c^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[c^g", "[^g", "[^g", "[^g"]; #ride 16 
    r[3,2] = ["[^g^D", "[^g", "[^g", "[^g", "[c^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[c^g^D", "[^g", "[^g", "[^g"]; #ride 16 with step beats
    r[4,2] = ["[^g", "[^g", "[^g^D", "[^g", "[c^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[c^g", "[^g", "[^g^D", "[^g" ]; #ride 16 with step ands
    r[1,3] = ["[g","[","[g","[","[cg","[","[g","[","[g","[","[g","[","[cg","[","[g","["] #bell 8
    r[2,3] = ["[g","[g","[","[","[cg","[g","[","[","[g","[g","[","[","[cg","[g","[","["] #bell 16th pairs on beat
    r[3,3] = ["[g","[^g","[g","[^g","[cg","[^g","[g","[^g","[g","[^g","[g","[^g","[cg","[^g","[g","[^g"] #bell drive, offbeat ride
    r[4,3] = ["[","[","[g","[g","[c","[","[g","[g","[","[","[g","[g","[c","[","[g","[g"] #bell pairs on and
    r[1,4] = ["[^D","[","[","[","[c^D","[","[","[","[^D","[","[","[","[c^D","[","[","["]  #step Q
    r[2,4] = ["[^D","[","[^D","[","[c^D","[","[^D","[","[^D","[","[^D","[","[c^D","[","[^D","["] #step 8
    r[3,4] = ["[g^D","[g","[^D","[","[cg^D","[g","[^D","[","[g^D","[g","[^D","[","[cg^D","[g","[^D","["] #step 8 with bell beat pair
    r[4,4] = ["[^D","[","[g^D","[g","[c^D","[","[g^D","[g","[^D","[","[g^D","[g","[c^D","[","[g^D","[g"] #step 8 with bell and pair
    return(r)
end


lookup16ride (generic function with 1 method)

In [78]:
#input a shiffled 16th bass rhythm and a shuffled 16th ride pattern, output abc notation
function abctime24patinst24bass(L, pattern, instr)
    n = length(L)
    r = lookup16ride()
    c = ["]/2", "F]/2"]
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        if i % 4 == 1
            s = s * "(6:4"
        end
        s = s * r[pattern, instr][(i - 1) % 16 + 1] * m[i]
        if i % 2 == 1 
 #           s = s * "Qc/2" #ghost snare
            s = s * "z/2" #rest
        elseif i % 4 == 0
            s = s * " "
        end
        if i % 16 == 0
            s = s * "|"
        end
    end
    return(s)
end


abctime24patinst24bass (generic function with 1 method)

In [79]:
#test
abctime24patinst24bass(bassgen(8, rockBass), 1, 1)

"(6:4N[^eF]/2z/2[]/2N[^e]/2z/2[]/2 (6:4N[c^eF]/2z/2[F]/2N[^e]/2z/2[]/2 (6:4N[^eF]/2z/2[]/2N[^e]/2z/2[]/2 (6:4N[c^eF]/2z/2[F]/2N[^e]/2z/2[]/2 |"

In [21]:
#output linear fill in $beats number of sixteenth triplets
function fill24gen(
  beats::Int,
  trans::AugDblMatrix)
    nextv = trans.init
    L = []
    for j = 1:beats
        current = markovrand(nextv)
        append!(L,current)
        if j % 2 == 1
            nextv = trans.mat1[1:6, current + 1]
        else
            nextv = trans.mat2[1:6, current + 1]
        end
    end
    return(L)
end


fill24gen (generic function with 1 method)

In [22]:
rockFillLeftToRight = [
    0.2e0 0.1e0 0.2e0 0 0.3e0 0.2e0; 
    0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 
    0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0 0.1e0 0.1e0 0]

6×6 Matrix{Float64}:
 0.2  0.1  0.2  0.0  0.3  0.2
 0.4  0.2  0.0  0.3  0.3  0.2
 0.1  0.4  0.2  0.1  0.1  0.2
 0.1  0.1  0.4  0.3  0.1  0.2
 0.1  0.1  0.2  0.2  0.1  0.2
 0.1  0.1  0.0  0.1  0.1  0.0

In [23]:
rockFillRightToLeft = [
    0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0
    0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0
    0.1e0 0 0.4e0 0.1e0 0.1e0 0
    0 0.1e0 0.1e0 0.4e0 0.1e0 0
    0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0
    0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.10e1]


6×6 Matrix{Float64}:
 0.4  0.2  0.1  0.1  0.3  0.0
 0.2  0.4  0.2  0.2  0.2  0.0
 0.1  0.0  0.4  0.1  0.1  0.0
 0.0  0.1  0.1  0.4  0.1  0.0
 0.2  0.1  0.1  0.1  0.2  0.0
 0.1  0.2  0.1  0.1  0.1  1.0

In [24]:
rockFillInitial = [0.4e0,0.4e0,0.1e0,0.1e0,0,0] #snare, high, mid, low, kick, rest

6-element Vector{Float64}:
 0.4
 0.4
 0.1
 0.1
 0.0
 0.0

In [25]:
rockFill= AugDblMatrix(rockFillLeftToRight, rockFillRightToLeft, rockFillInitial)

AugDblMatrix([0.2 0.1 … 0.3 0.2; 0.4 0.2 … 0.3 0.2; … ; 0.1 0.1 … 0.1 0.2; 0.1 0.1 … 0.1 0.0], [0.4 0.2 … 0.3 0.0; 0.2 0.4 … 0.2 0.0; … ; 0.2 0.1 … 0.2 0.0; 0.1 0.2 … 0.1 1.0], [0.4, 0.4, 0.1, 0.1, 0.0, 0.0])

In [26]:
#input a sixteenth triplet fill and a crescendo/descr indicator, output abc notation
function abcfill24(L, crdc)
    n = length(L)
    c = ["c/2","e/2","A/2","G/2","F/2","z/2"]
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        if i % 6 == 1
            s = s * "(6:4"
        end
        s = s * m[i]
        if i % 6 == 0
            s = s * " "
        end
        if i % 24 == 16
            if crdc < 0
                s = s * "!>(!"
            elseif 0 < crdc
                s = s * "!<(!"
            end
        end
        if i % 24 == 20
            if crdc < 0
                s = s * "!>)!"
            elseif 0 < crdc
                s = s * "!<)!"
            end
        end
    end
    return(s)
end

abcfill24 (generic function with 1 method)

In [27]:
#test
abcfill24(fill24gen(24, rockFill), 0)

"(6:4e/2G/2G/2F/2e/2e/2 (6:4z/2A/2c/2e/2G/2F/2 (6:4F/2A/2A/2A/2c/2e/2 (6:4e/2A/2z/2G/2e/2A/2 "

In [28]:
#output time, fill, time, fill with cresc/decr in abc notation
function abctune16shuffle16bass(
  vol,
  bass,
  pat,
  inst,
  fill,
  crdc)
    fp = ["!pp!","!p!","!mp!","!mf!","!f!","!ff!"]
    return("|:" * fp[vol] * abctime24patinst24bass(bass, pat, inst) * "[1" * abcfill24(fill, 0) * ":|2" * abcfill24(fill, crdc))
end


abctune16shuffle16bass (generic function with 1 method)

In [29]:
#utility function, insert one string into another
function insert(
    s::String,
    pos::Integer,
    sub::String)
    return s[1:pos] * sub * s[pos+1:end]
end
    

insert (generic function with 1 method)

In [30]:
#test
insert("abcdefg", 4, "x")

"abcdxefg"

In [31]:
macro functionName()
    return quote
        st = stacktrace(backtrace())
        myf = ""
        for frm in st
            funcname = frm.func
            if frm.func != :backtrace && frm.func!= Symbol("macro expansion")
                myf = frm.func
                break
            end
        end
        #println("Running function $(myf)")
        String(myf)
    end
end


@functionName (macro with 1 method)

In [58]:
#put it all together, output abc notation
import Dates
function abcsolo16shuffle16bass(
  len::Int,
  basstrans::AugMatrix,
  filltrans::AugDblMatrix,
  dyntrans::AugDblMatrix,
  pattrans::AugMatrix,
  insttrans::AugMatrix,
  feel::String,
    )
    s = ""
    curdyn = markovrand(dyn.init)
    curpat = markovrand(pattrans.init)
    curinst = markovrand(insttrans.init)
    for i = 1:len
        vol = div(curdyn, 2) + 1
        bass = bassgen(24, basstrans)
        pat = curpat + 1
        inst = curinst + 1
        fill = fill24gen(24, filltrans)
        crdc = 2 * mod(curdyn, 2) - 1
        s = s * abctune16shuffle16bass(vol, bass, pat, inst, fill, crdc)
        if i + 1 < len
            curdyn = markovrand(dyntrans.mat1[:, curdyn + 1])
        else
            curdyn = markovrand(dyntrans.mat2[:, curdyn + 1])
        end
        curpat = markovrand(pattrans.mat[:, curpat + 1])
        curinst = markovrand(insttrans.mat[:, curinst + 1])
    end
    s = insert(s, findlast(occursin("acbdefgABCDEFG"), s)-1, "H") #final fermata
    s = replace(s, "[]" => "z") #rest
    s = replace(s, "!>!" => "!>![I: volinc 50]") #accent
    s = replace(s, "U" => "!anti![I: volinc -20]") #unaccent
    s = replace(s, "N" => "\"^_\"") #tenuto
    s = replace(s, "Q" => "\"<(\"\">)\"[I:volinc -50]") #ghost    
    s = makeheader("Computer-generated \\`Etude for Solo Drum Kit", String(@functionName), "$(Dates.now())", "4/4", feel, 60) * s * "|]\n"
    return(s)
end


abcsolo16shuffle16bass (generic function with 1 method)

In [59]:
function makeheader(title::String, composer::String, date::String, meter::String, feel::String, tempo::Int)
    s = """%%abc-include percussions-JBH.abh
    
    %%flatbeams
    %%propagate-accidentals not
    %%pos ornament up
    %%ornament up
    %%MIDI fermatafixed
    U: N = !tenuto!
    U: U = !anti!
    U: Q = !ghost! %doesn't exist but it should!
    
    X:1
    T:$title
    C:$composer
    O:$date
    M:$meter
    L:1/8
    Q:\"$feel\" 1/4=$tempo
    K:none clef=perc
    [V:1 clef=perc, stem=up]     % activate abc2xml.py map
    %%voicemap drummap  % activate abcm2ps/abc2svg map
    %%MIDI channel 10   % activate abc2midi map
    %%MIDI program 0
    """
    return(s)
end

makeheader (generic function with 1 method)

In [60]:
#test
print(makeheader("Computer-generated \\`Etude for Solo Drum Kit", String(@functionName), "$(Dates.now())", "4/4", "Rock 16th feel", 80))

%%abc-include percussions-JBH.abh

%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
U: N = !tenuto!
U: U = !anti!
U: Q = !ghost! %doesn't exist but it should!

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:top-level scope
O:2021-11-11T19:31:12.277
M:4/4
L:1/8
Q:"Rock 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0


In [61]:
#write a string s of abc notation to a file fname
function writetune(fname, s)
    #h = open("header.abc", "r")
    f = open(fname, "w")
    #for header in eachline(h)
    #    print(f, header * "\n")
    #end
    #close(h)
    #print(f, makeheader("Computer-generated \\`Etude for Solo Drum Kit", "4/4", "Rock 16th feel", 80))
    print(f, s)
    return(close(f))
end


writetune (generic function with 1 method)

In [62]:
dynMatrix = [
    0 0 0.2e0 0.2e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.1e0 0.2e0; 
    0 0 0.2e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.2e0 0.1e0; 
    0.1e0 0.2e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.2e0 0 0; 
    0.2e0 0.1e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0;
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.2e0 0.2e0 0 0]


12×12 Matrix{Float64}:
 0.0  0.0  0.2  0.2  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.2  0.1  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.1  0.2  0.1  0.2
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.2  0.1  0.2  0.1
 0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.1  0.2
 0.0  0.0  0.2  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.2  0.1
 0.1  0.2  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.2  0.0  0.0
 0.2  0.1  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0
 0.1  0.2  0.1  0.2  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.2  0.1  0.2  0.1  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.1  0.2  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.2  0.2  0.0  0.0

In [63]:
dynFinal = [
    0.1e0 0.2e0 0.2e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.2e0 0.2e0 0.1e0]


12×12 Matrix{Float64}:
 0.1  0.2  0.2  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.0  0.0
 0.2  0.1  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.1  0.0  0.0
 0.2  0.2  0.0  0.0  0.2  0.1  0.2  0.1  0.2  0.2  0.1  0.2
 0.2  0.2  0.0  0.0  0.2  0.1  0.2  0.1  0.2  0.1  0.2  0.1
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.1  0.2  0.1  0.2  0.1  0.2  0.1  0.2  0.0  0.0  0.2  0.2
 0.2  0.1  0.2  0.2  0.1  0.2  0.1  0.2  0.0  0.0  0.2  0.2
 0.0  0.0  0.1  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.1  0.2
 0.0  0.0  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.2  0.2  0.1

In [64]:
dynInitial = [0.5e-1,0.5e-1,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.5e-1,0.5e-1]
#pp>, pp<, p>, p<, mp>, mp<, mf>, mf<, f>, f<, ff>, ff<

12-element Vector{Float64}:
 0.05
 0.05
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.05
 0.05

In [65]:
dyn = AugDblMatrix(dynMatrix, dynFinal, dynInitial)

AugDblMatrix([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.1 0.2 … 0.0 0.0; 0.2 0.1 … 0.0 0.0; … ; 0.0 0.0 … 0.1 0.2; 0.0 0.0 … 0.2 0.1], [0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05])

In [66]:
rockPatMatrix = [
    #8  Q   b   +
    0   0.4 0.4 0.4 #8 
    0.4 0   0.2 0.2   #Q
    0.2 0.2 0   0.4 #b
    0.4 0.4 0.4 0 #+
    ]



4×4 Matrix{Float64}:
 0.0  0.4  0.4  0.4
 0.4  0.0  0.2  0.2
 0.2  0.2  0.0  0.4
 0.4  0.4  0.4  0.0

In [67]:
sum(rockPatMatrix, dims=1)

1×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0

In [68]:
rockPatInitial = [0.4; 0.4; 0.2; 0]

4-element Vector{Float64}:
 0.4
 0.4
 0.2
 0.0

In [69]:
rockPat = AugMatrix(rockPatMatrix, rockPatInitial)

AugMatrix([0.0 0.4 0.4 0.4; 0.4 0.0 0.2 0.2; 0.2 0.2 0.0 0.4; 0.4 0.4 0.4 0.0], [0.4, 0.4, 0.2, 0.0])

In [70]:
rockInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

4×4 Matrix{Float64}:
 0.1  0.4  0.4  0.4
 0.6  0.1  0.4  0.4
 0.2  0.4  0.1  0.2
 0.1  0.1  0.1  0.0

In [71]:
sum(rockInstMatrix, dims=1)

1×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0

In [72]:
rockInstInitial = [0.5; 0.3; 0; 0.2]

4-element Vector{Float64}:
 0.5
 0.3
 0.0
 0.2

In [73]:
rockInst = AugMatrix(rockInstMatrix, rockInstInitial)

AugMatrix([0.1 0.4 0.4 0.4; 0.6 0.1 0.4 0.4; 0.2 0.4 0.1 0.2; 0.1 0.1 0.1 0.0], [0.5, 0.3, 0.0, 0.2])

In [80]:
solo = abcsolo16shuffle16bass(4, rockBass, rockFill, dyn, rockPat, rockInst, "Rock 16th shuffle feel")
print(solo)

%%abc-include percussions-JBH.abh

%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
U: N = !tenuto!
U: U = !anti!
U: Q = !ghost! %doesn't exist but it should!

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16shuffle16bass
O:2021-11-11T19:36:23.337
M:4/4
L:1/8
Q:"Rock 16th shuffle feel" 1/4=60
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
|:!ff!(6:4[^DF]/2z/2z/2[F]/2z/2[F]/2 (6:4[c^D]/2z/2z/2[F]/2z/2[F]/2 (6:4[^D]/2z/2z/2[F]/2z/2z/2 (6:4[c^DF]/2z/2z/2[F]/2z/2[F]/2 |(6:4[^D]/2z/2z/2[F]/2z/2z/2 (6:4[c^D]/2z/2z/2z/2z/2z/2 (6:4[^D]/2z/2z/2[F]/2z/2z/2 (6:4[c^DF]/2z/2z/2z/2z/2z/2 |(6:4[^DF]/2z/2z/2[F]/2z/2z/2 (6:4[c^DF]/2z/2z/2[F]/2z/2[F]/2 (6:4[^D]/2z/2z/2[F]/2z/2[F]/2 (6:4[c^D]/2z/2z/2z/2z/2z/2 |[1(6:4c/2G/2e/2A/2e/2z/2 (6:4z/2G/2G/2G/2F/2G/2 (6:4e/2A/2z/2c/2A/2G/2 (6:4c/2A/2e/2e/2e/2A/2 :|2(6:4c/2G

In [81]:
writetune("rocktest.abc", solo)

In [82]:
run(`cat rocktest.abc`)

%%abc-include percussions-JBH.abh

%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
U: N = !tenuto!
U: U = !anti!
U: Q = !ghost! %doesn't exist but it should!

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16shuffle16bass
O:2021-11-11T19:36:23.337
M:4/4
L:1/8
Q:"Rock 16th shuffle feel" 1/4=60
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
|:!ff!(6:4[^DF]/2z/2z/2[F]/2z/2[F]/2 (6:4[c^D]/2z/2z/2[F]/2z/2[F]/2 (6:4[^D]/2z/2z/2[F]/2z/2z/2 (6:4[c^DF]/2z/2z/2[F]/2z/2[F]/2 |(6:4[^D]/2z/2z/2[F]/2z/2z/2 (6:4[c^D]/2z/2z/2z/2z/2z/2 (6:4[^D]/2z/2z/2[F]/2z/2z/2 (6:4[c^DF]/2z/2z/2z/2z/2z/2 |(6:4[^DF]/2z/2z/2[F]/2z/2z/2 (6:4[c^DF]/2z/2z/2[F]/2z/2[F]/2 (6:4[^D]/2z/2z/2[F]/2z/2[F]/2 (6:4[c^D]/2z/2z/2z/2z/2z/2 |[1(6:4c/2G/2e/2A/2e/2z/2 (6:4z/2G/2G/2G/2F/2G/2 (6:4e/2A/2z/2c/2A/2G/2 (6:4c/2A/2e/2e/2e/2A/2 :|2(6:4c/2G

Process(`cat rocktest.abc`, ProcessExited(0))

In [83]:
run(`abctest.bat rocktest.abc`)


(base) C:\Users\holden\Documents\GitHub\drum_solo_generator>abcm2ps.exe -X rocktest.abc -O rocktest.xhtml 
abcm2ps-8.14.12 (2021-07-14)
File rocktest.abc
File percussions-JBH.abh
Output written on rocktest.xhtml (1 page, 1 title, 75961 bytes)

(base) C:\Users\holden\Documents\GitHub\drum_solo_generator>abc2midi.exe rocktest.abc -o rocktest.mid 
4.59 June 27 2021 abc2midi
Warning in line-char 24-17 : Different length notes in tuple
Warning in line-char 24-20 : Different length notes in tuple
Warning in line-char 24-28 : Different length notes in tuple
Warning in line-char 24-48 : Different length notes in tuple
Warning in line-char 24-51 : Different length notes in tuple
Warning in line-char 24-59 : Different length notes in tuple
Warning in line-char 24-78 : Different length notes in tuple
Warning in line-char 24-81 : Different length notes in tuple
Warning in line-char 24-89 : Different length notes in tuple
Warning in line-char 24-92 : Different length notes in tuple
Warning in line

rocktest.abc:24:318: warning: Line overfull (637pt of 628pt)



(base) C:\Users\holden\Documents\GitHub\drum_solo_generator>start rocktest.xhtml 


Process(`abctest.bat rocktest.abc`, ProcessExited(0))

In [84]:
funkBassMatrix = [0.1e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0; 0.3e0 0 0.25e0 0]

4×4 Matrix{Float64}:
 0.1  0.333333  0.25  0.5
 0.3  0.333333  0.25  0.5
 0.3  0.333333  0.25  0.0
 0.3  0.0       0.25  0.0

In [85]:
funkBassInitial = [0.1e0,0.1e0,0.4e0,0.4e0]

4-element Vector{Float64}:
 0.1
 0.1
 0.4
 0.4

In [138]:
funkBass = AugMatrix(funkBassMatrix, funkBassInitial)

AugMatrix([0.1 0.3333333333333333 0.25 0.5; 0.3 0.3333333333333333 0.25 0.5; 0.3 0.3333333333333333 0.25 0.0; 0.3 0.0 0.25 0.0], [0.1, 0.1, 0.4, 0.4])

In [139]:
funkFillLeftToRight = [0.2e0 0.1e0 0.1e0 0 0.3e0 0.2e0; 0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0; 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0]

6×6 Matrix{Float64}:
 0.2  0.1  0.1  0.0  0.3  0.2
 0.4  0.2  0.0  0.3  0.3  0.2
 0.1  0.4  0.2  0.1  0.1  0.2
 0.1  0.1  0.4  0.3  0.1  0.2
 0.1  0.1  0.1  0.2  0.1  0.1
 0.1  0.1  0.2  0.1  0.1  0.1

In [140]:
funkFillRightToLeft = [0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0.2e0; 0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0.2e0; 0.1e0 0 0.4e0 0.1e0 0.1e0 0.2e0; 0 0.1e0 0.1e0 0.4e0 0.1e0 0.2e0; 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0; 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0]


6×6 Matrix{Float64}:
 0.4  0.2  0.1  0.1  0.3  0.2
 0.2  0.4  0.2  0.2  0.2  0.2
 0.1  0.0  0.4  0.1  0.1  0.2
 0.0  0.1  0.1  0.4  0.1  0.2
 0.2  0.1  0.1  0.1  0.2  0.1
 0.1  0.2  0.1  0.1  0.1  0.1

In [141]:
funkFillInitial = [0.3e0,0.3e0,0.1e0,0.1e0,0.1e0,0.1e0]

6-element Vector{Float64}:
 0.3
 0.3
 0.1
 0.1
 0.1
 0.1

In [142]:
funkFill = AugDblMatrix(funkFillLeftToRight, funkFillRightToLeft, funkFillInitial)

AugDblMatrix([0.2 0.1 … 0.3 0.2; 0.4 0.2 … 0.3 0.2; … ; 0.1 0.1 … 0.1 0.1; 0.1 0.1 … 0.1 0.1], [0.4 0.2 … 0.3 0.2; 0.2 0.4 … 0.2 0.2; … ; 0.2 0.1 … 0.2 0.1; 0.1 0.2 … 0.1 0.1], [0.3, 0.3, 0.1, 0.1, 0.1, 0.1])

In [143]:
funkRideMatrix = [
     #HH8 #RQ    #R8sB #R8sA #beA  #sl4A #sl2A #sl1A #HHdr8 #s8beB
    0 0.1e0 0.1e0 0.1e0 0.2e0 0.2e0 0.2e0 0.2e0 0 0.2e0; #HH8 
    0.2e0 0 0.2e0 0.2e0 0.1e0 0.2e0 0.2e0 0.2e0 0.2e0 0.1e0; #RQ   
    0.1e0 0.1e0 0 0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0;   #R8sB 
    0.1e0 0.1e0 0 0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0;  #R8sA 
    0.2e0 0.2e0 0.2e0 0.2e0 0 0.1e0 0.1e0 0.1e0 0.2e0 0; #beA 
    0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0 0.1e0 0.1e0 0.1e0 0.1e0;  #sl4A
    0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0 0 0 0.1e0 0.1e0; #sl2A
    0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0 0 0 0.1e0 0.1e0; #sl1A
    0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0 0.2e0; #HHdr8
    0.1e0 0.1e0 0.1e0 0.1e0 0 0.2e0 0.1e0 0.1e0 0.1e0 0 #s8beB
]


10×10 Matrix{Float64}:
 0.0  0.1  0.1  0.1  0.2  0.2  0.2  0.2  0.0  0.2
 0.2  0.0  0.2  0.2  0.1  0.2  0.2  0.2  0.2  0.1
 0.1  0.1  0.0  0.0  0.1  0.1  0.1  0.1  0.1  0.1
 0.1  0.1  0.0  0.0  0.1  0.1  0.1  0.1  0.1  0.1
 0.2  0.2  0.2  0.2  0.0  0.1  0.1  0.1  0.2  0.0
 0.1  0.1  0.1  0.1  0.1  0.0  0.1  0.1  0.1  0.1
 0.1  0.1  0.1  0.1  0.1  0.0  0.0  0.0  0.1  0.1
 0.1  0.1  0.1  0.1  0.1  0.0  0.0  0.0  0.1  0.1
 0.0  0.1  0.1  0.1  0.2  0.1  0.1  0.1  0.0  0.2
 0.1  0.1  0.1  0.1  0.0  0.2  0.1  0.1  0.1  0.0

In [144]:
funkRideInitial = [0.2e0,0.2e0,0,0,0.1e0,0.1e0,0.1e0,0.1e0,0.2e0,0]

10-element Vector{Float64}:
 0.2
 0.2
 0.0
 0.0
 0.1
 0.1
 0.1
 0.1
 0.2
 0.0

In [145]:
funkRide = AugMatrix(funkRideMatrix, funkRideInitial)

AugMatrix([0.0 0.1 … 0.0 0.2; 0.2 0.0 … 0.2 0.1; … ; 0.0 0.1 … 0.0 0.2; 0.1 0.1 … 0.1 0.0], [0.2, 0.2, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.2, 0.0])

In [170]:
funkPatMatrix = [
    #8  Q   b   +
    0   0.2 0.2 0.2 #8 
    0.2 0   0.4 0.4   #Q
    0.2 0.2 0   0.4 #b
    0.6 0.6 0.4 0 #+
    ]

4×4 Matrix{Float64}:
 0.0  0.2  0.2  0.2
 0.2  0.0  0.4  0.4
 0.2  0.2  0.0  0.4
 0.6  0.6  0.4  0.0

In [171]:
sum(funkPatMatrix, dims=1)

1×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0

In [172]:
funkPatInitial = [0; 0.2; 0.4; 0.4]

4-element Vector{Float64}:
 0.0
 0.2
 0.4
 0.4

In [173]:
funkPat = AugMatrix(funkPatMatrix, funkPatInitial)

AugMatrix([0.0 0.2 0.2 0.2; 0.2 0.0 0.4 0.4; 0.2 0.2 0.0 0.4; 0.6 0.6 0.4 0.0], [0.0, 0.2, 0.4, 0.4])

In [174]:
funkInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

4×4 Matrix{Float64}:
 0.1  0.4  0.4  0.4
 0.6  0.1  0.4  0.4
 0.2  0.4  0.1  0.2
 0.1  0.1  0.1  0.0

In [175]:
funkInstInitial = [0.4; 0.4; 0.2; 0]

4-element Vector{Float64}:
 0.4
 0.4
 0.2
 0.0

In [176]:
funkInst = AugMatrix(funkInstMatrix, funkInstInitial)

AugMatrix([0.1 0.4 0.4 0.4; 0.6 0.1 0.4 0.4; 0.2 0.4 0.1 0.2; 0.1 0.1 0.1 0.0], [0.4, 0.4, 0.2, 0.0])

In [177]:
funksolo = abcsolo8patinst16bass(4, funkBass, funkFill, dyn, funkPat, funkInst, "Funk 16th feel")
print(funksolo)

%%abc-include percussions-JBH.abh

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo8patinst16bass
O:2021-10-19T18:55:34.504
M:4/4
L:1/8
Q:"Funk 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!f![gF]/2[F/2][^g]y[cg]y[^gF]/2[F/2] [g]/2[F/2][^g]y[cg]/2[F/2][^gF]y |[g]/2[F/2][^gF]y[cg]/2[F/2][^g]y [gF]/2[F/2][^g]/2[F/2][cgF]y[^gF]y |[g]/2[F/2][^gF]y[cgF]y[^gF]/2[F/2] [g]y[^g]/2[F/2][cg]y[^g]y |[1F/2A/2e/2F/2 z/2F/2G/2G/2 A/2G/2G/2e/2 c/2A/2c/2F/2 :|2F/2A/2e/2F/2 z/2F/2G/2G/2 A/2G/2G/2e/2 !<(!c/2A/2!<)!c/2F/2 |:!mf![F]y[gF]y[c]y[gF]/2[F/2] z/2[F/2][g]/2[F/2][c]/2[F/2][g]/2[F/2] |zy[gF]y[c]y[gF]y z/2[F/2][g]/2[F/2][cF]y[gF]y |zy[gF]y[cF]/2[F/2][g]y [F]y[g]/2[F/2][c]y[gF]/2[F/2] |[1c/2F/2e/2z/2 c/2e/2e/2A/2 A/2G/2z/2A/2 A/2z/2c/2e/2 :|2

In [178]:
writetune("funktest.abc", funksolo)

In [179]:
run(`cat funktest.abc`)

%%abc-include percussions-JBH.abh

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo8patinst16bass
O:2021-10-19T18:55:34.504
M:4/4
L:1/8
Q:"Funk 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!f![gF]/2[F/2][^g]y[cg]y[^gF]/2[F/2] [g]/2[F/2][^g]y[cg]/2[F/2][^gF]y |[g]/2[F/2][^gF]y[cg]/2[F/2][^g]y [gF]/2[F/2][^g]/2[F/2][cgF]y[^gF]y |[g]/2[F/2][^gF]y[cgF]y[^gF]/2[F/2] [g]y[^g]/2[F/2][cg]y[^g]y |[1F/2A/2e/2F/2 z/2F/2G/2G/2 A/2G/2G/2e/2 c/2A/2c/2F/2 :|2F/2A/2e/2F/2 z/2F/2G/2G/2 A/2G/2G/2e/2 !<(!c/2A/2!<)!c/2F/2 |:!mf![F]y[gF]y[c]y[gF]/2[F/2] z/2[F/2][g]/2[F/2][c]/2[F/2][g]/2[F/2] |zy[gF]y[c]y[gF]y z/2[F/2][g]/2[F/2][cF]y[gF]y |zy[gF]y[cF]/2[F/2][g]y [F]y[g]/2[F/2][c]y[gF]/2[F/2] |[1c/2F/2e/2z/2 c/2e/2e/2A/2 A/2G/2z/2A/2 A/2z/2c/2e/2 :|2

Process(`cat funktest.abc`, ProcessExited(0))

In [180]:
run(`abctest.bat funktest.abc`)


(base) C:\Users\holden\Documents\GitHub\drum_solo_generator>abcm2ps.exe -X funktest.abc -O funktest.xhtml 
abcm2ps-8.14.12 (2021-07-14)
File funktest.abc
File percussions-JBH.abh
Output written on funktest.xhtml (1 page, 1 title, 46991 bytes)

(base) C:\Users\holden\Documents\GitHub\drum_solo_generator>abc2midi.exe funktest.abc -o funktest.mid 
4.59 June 27 2021 abc2midi
writing MIDI file funktest.mid
Warning in line-char 20-1312 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat

(base) C:\Users\holden\Documents\GitHub\drum_solo_generator>start funktest.mid 


funktest.abc:20:245: warning: Line overfull (630pt of 628pt)



(base) C:\Users\holden\Documents\GitHub\drum_solo_generator>start funktest.xhtml 


Process(`abctest.bat funktest.abc`, ProcessExited(0))